In [ ]:
import numpy as np
import pandas as pd

import folium
from folium import Marker
from folium.plugins import MarkerCluster

In [ ]:
ev_stations_df = pd.read_csv('/content/ev_stations_v1.csv')
ev_stations_df.head()

<ipython-input-5-d52259811c8b>:1: DtypeWarning: Columns (20,31,46) have mixed types. Specify dtype option on import or set low_memory=False.
  ev_stations_df = pd.read_csv('/content/ev_stations_v1.csv')


,Fuel Type Code,Station Name,Street Address,Intersection Directions,City,State,ZIP,Plus4,Station Phone,Status Code,...,EV Pricing (French),LPG Nozzle Types,Hydrogen Pressures,Hydrogen Standards,CNG Fill Type Code,CNG PSI,CNG Vehicle Class,LNG Vehicle Class,EV On-Site Renewable Source,Restricted Access
0,ELEC,LADWP - Truesdale Center,11797 Truesdale St,NaN,Sun Valley,CA,91352,NaN,NaN,E,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,ELEC,LADWP - West LA District Office,1394 S Sepulveda Blvd,NaN,Los Angeles,CA,90024,NaN,NaN,E,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,ELEC,Los Angeles Convention Center,1201 S Figueroa St,West hall and South hall,Los Angeles,CA,90015,NaN,213-741-1151,E,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,ELEC,LADWP - John Ferraro Building,111 N Hope St,Across Hope,Los Angeles,CA,90012,NaN,NaN,E,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,ELEC,LADWP - Haynes Power Plant,6801 E 2nd St,NaN,Long Beach,CA,90803,NaN,NaN,E,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
ev_stations_df.columns

Index(['Fuel Type Code', 'Station Name', 'Street Address',
       'Intersection Directions', 'City', 'State', 'ZIP', 'Plus4',
       'Station Phone', 'Status Code', 'Expected Date',
       'Groups With Access Code', 'Access Days Time', 'Cards Accepted',
       'BD Blends', 'NG Fill Type Code', 'NG PSI', 'EV Level1 EVSE Num',
       'EV Level2 EVSE Num', 'EV DC Fast Count', 'EV Other Info', 'EV Network',
       'EV Network Web', 'Geocode Status', 'Latitude', 'Longitude',
       'Date Last Confirmed', 'ID', 'Updated At', 'Owner Type Code',
       'Federal Agency ID', 'Federal Agency Name', 'Open Date',
       'Hydrogen Status Link', 'NG Vehicle Class', 'LPG Primary',
       'E85 Blender Pump', 'EV Connector Types', 'Country',
       'Intersection Directions (French)', 'Access Days Time (French)',
       'BD Blends (French)', 'Groups With Access Code (French)',
       'Hydrogen Is Retail', 'Access Code', 'Access Detail Code',
       'Federal Agency Code', 'Facility Type', 'CNG Dispenser N

In [ ]:
# 데이터프레임에서 'Latitude'와 'Longitude' 열이 모두 null이 아닌 행만 선택
locations_df = ev_stations_df[ev_stations_df['Latitude'].notnull() & ev_stations_df['Longitude'].notnull()]

# 'State' 열이 'NC'이고 'City' 열이 'Charlotte'인 행만 선택 -> 노스캐롤라이나 주(Charlotte)의 충전소 위치 데이터만을 추출
Charlotte_NC_locations_df = locations_df[(locations_df['State'] == 'NC') & (locations_df['City'] == 'Charlotte')]
Charlotte_NC_locations_df.head()

,Fuel Type Code,Station Name,Street Address,Intersection Directions,City,State,ZIP,Plus4,Station Phone,Status Code,...,EV Pricing (French),LPG Nozzle Types,Hydrogen Pressures,Hydrogen Standards,CNG Fill Type Code,CNG PSI,CNG Vehicle Class,LNG Vehicle Class,EV On-Site Renewable Source,Restricted Access
640,ELEC,CLT AIRPORT BV2 BASEMENT 1,5601 Wilkinson Blvd,NaN,Charlotte,NC,28208,NaN,888-758-4389,E,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
730,ELEC,The Green Parking Garage,435 S Tryon St,At S Tryon St and Levine Avenue of the Arts,Charlotte,NC,28202,NaN,604-338-3232,E,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1188,ELEC,"CHARLOTTE, NC CMGC STATION 4",232 S Davidson St,NaN,Charlotte,NC,28202,NaN,888-758-4389,E,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1189,ELEC,"CHARLOTTE, NC MALLARD CK 02",1712 J N Pease Pl,NaN,Charlotte,NC,28262,NaN,888-758-4389,E,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1627,ELEC,East Charlotte Nissan,6901 E Independence Blvd,NaN,Charlotte,NC,28227,NaN,704-535-4012,E,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# 지도 만들기
charlotte_map = folium.Map(location=[35.227,-80.843], tiles='openstreetmap', zoom_start=11)

# 지도에 포인트 추가하기
for idx, row in Charlotte_NC_locations_df.iterrows():
    Marker(location=[row['Latitude'], row['Longitude']],
           popup=row['Street Address']).add_to(charlotte_map)

# 지도 노출
charlotte_map

In [ ]:
NC_locations_df = locations_df[(locations_df['State'] == 'NC')]
NC_locations_df.head()

In [ ]:
# Create a map centered on North Carolina
nc_map = folium.Map(location=[35.5908438,-79.7628341], tiles='openstreetmap', zoom_start=7)

# Add points to a marker cluster
mc = MarkerCluster()
for idx, row in NC_locations_df.iterrows():
    mc.add_child(Marker(location=[row['Latitude'], row['Longitude']],
                        popup=row['Street Address']))

# add the marker cluster to the map
nc_map.add_child(mc)

# Display the map
nc_map

In [ ]:
# 한국기준 테스트

# 지도 객체 생성
map = folium.Map(location=[37.5665, 126.9780], zoom_start=13)

# 마커 추가
folium.Marker(location=[37.5665, 126.9780], popup='서울특별시청').add_to(map)
folium.Marker(location=[37.5645, 126.9750], popup='덕수궁').add_to(map)

# 지도 출력
map.save('map.html')

In [ ]:
import re

bokgu_ev_stations_df = pd.read_csv('/content/bokgu.csv', encoding='cp949')

# 'Street Address'안에 있는 괄호 영역 삭제
bokgu_ev_stations_df['Station Name'] = bokgu_ev_stations_df['Station Name'].apply(lambda x: re.sub(r'\([^()]*\)', '', x))
bokgu_ev_stations_df['Street Address'] = bokgu_ev_stations_df['Street Address'].apply(lambda x: re.sub(r'\([^()]*\)', '', x))

bokgu_ev_stations_df.head()

,Station Name,Street Address,Latitude,Longitude,Date Last Confirmed
0,대망주유소,대구광역시 북구 중앙대로 601,35.886578,128.597635,2023-06-22
1,아스트로 주유소,대구광역시 북구 팔달로 215,35.888247,128.573208,2023-06-22
2,대성산업대성주유소,대구광역시 북구 침산로 62,35.882038,128.590080,2023-06-22
3,해바라기Self주유소,대구광역시 북구 대현로 81,35.883114,128.608069,2023-06-22
4,SK에너지한일주유소,대구광역시 북구 칠곡중앙대로 315,35.923828,128.546552,2023-06-22


In [ ]:
# 데이터 프레임 생성 테스트
data = {
    'Station Name': ['대망주유소', '(주)아스트로 주유소', '대성산업(주)대성주유소', '해바라기Self주유소', 'SK에너지(주)한일주유소'],
    'Street Address': ['대구광역시 북구 중앙대로 601 (침산동)', '대구광역시 북구 팔달로 215 (노원동2가)', '대구광역시 북구 침산로 62 (칠성동2가)', '대구광역시 북구 대현로 81 (대현동)', '대구광역시 북구 칠곡중앙대로 315 (태전동)'],
    'Latitude': [35.886578, 35.888247, 35.882038, 35.883114, 35.923828],
    'Longitude': [128.597635, 128.573208, 128.590080, 128.608069, 128.546552],
    'Date Last Confirmed': ['2023-06-22', '2023-06-22', '2023-06-22', '2023-06-22', '2023-06-22']
}

df = pd.DataFrame(data)

# 지도 객체 생성
map = folium.Map(location=[35.883114, 128.597635], zoom_start=12)

# 데이터프레임 반복하면서 마커 추가
for index, row in df.iterrows():
    folium.Marker(
        location=[row['Latitude'], row['Longitude']],
        popup=row['Station Name']
    ).add_to(map)

# 지도 출력
map

In [ ]:
bokgu_ev_stations_df.columns

Index(['Station Name', 'Street Address', 'Latitude', 'Longitude',
       'Date Last Confirmed'],
      dtype='object')

In [ ]:
# 전체 데이터 프레임 생성
df = pd.DataFrame(bokgu_ev_stations_df)

# 지도 객체 생성
map = folium.Map(location=[35.883114, 128.597635], zoom_start=12)

# 데이터프레임 반복하면서 마커 추가
for index, row in df.iterrows():
    folium.Marker(
        location=[row['Latitude'], row['Longitude']],
        popup=row['Station Name']
    ).add_to(map)

# 지도 출력
map